<a href="https://colab.research.google.com/github/IrinaM21/leaf-classification/blob/main/LeafClassification_IrinaM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Below is my solution to my task, which involves the classification of leaf images and is based on the UCI leaf dataset.

I'll start by importing some necessary libraries.

In [ ]:
#import
import os
import zipfile
import tensorflow as tf
from google.colab import drive
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Next, I'll form the directories I'll be using to separate and label my data.

In [ ]:
drive.mount('/content/drive/')

train_leaf = os.path.join('/content/drive/My Drive/LeafImages')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Then, I'll design my neural net. 

In [ ]:
model = tf.keras.Sequential([
                             tf.keras.layers.Conv2D(16,(3,3),activation = 'relu', input_shape = (150, 150, 3)),
                             tf.keras.layers.MaxPool2D((2,2)),
                             tf.keras.layers.Conv2D(32,(3,3), activation = 'relu'),
                             tf.keras.layers.MaxPool2D((2,2)),
                             tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
                             tf.keras.layers.MaxPool2D((2,2)),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(512, activation = 'relu'),
                             tf.keras.layers.AlphaDropout(0.5),
                             tf.keras.layers.Dense(200, activation = 'relu'),
                             tf.keras.layers.AlphaDropout(0.5),
                             tf.keras.layers.Dense(40, activation = 'softmax')                   
])       

Next, I'll preprocess my data.

In [ ]:
train_leaf = ImageDataGenerator(rescale=1/255, validation_split=0.09)

train_generator = train_leaf.flow_from_directory(
        '/content/drive/My Drive/LeafImages/',
        target_size=(150, 150),
        batch_size=30,
        class_mode='sparse',
        subset= 'training')

validation_generator = train_leaf.flow_from_directory(
        '/content/drive/My Drive/LeafImages/',
        target_size=(150, 150),
        batch_size=10,
        class_mode='sparse',
        subset= 'validation')

Found 434 images belonging to 40 classes.
Found 19 images belonging to 40 classes.


I'll then compile the model to find the loss function.

In [ ]:
model.compile(
    loss = "sparse_categorical_crossentropy",
    optimizer=RMSprop(lr=0.001),
    metrics=['acc']
)

I'll check my model using model.summary()

In [ ]:
model.build()

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 18496)            

I'll also train my NN.

In [ ]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=8,  
      epochs=500,
      verbose=1)

Epoch 1/500
8/8 [==============================] - 3s 418ms/step - loss: 4.3918 - acc: 0.0155
Epoch 2/500
8/8 [==============================] - 3s 384ms/step - loss: 4.5836 - acc: 0.0307
Epoch 3/500
8/8 [==============================] - 3s 354ms/step - loss: 4.4252 - acc: 0.0206
Epoch 4/500
8/8 [==============================] - 3s 393ms/step - loss: 4.5155 - acc: 0.0396
Epoch 5/500
8/8 [==============================] - 3s 392ms/step - loss: 4.3849 - acc: 0.0185
Epoch 6/500
8/8 [==============================] - 3s 330ms/step - loss: 4.4067 - acc: 0.0261
Epoch 7/500
8/8 [==============================] - 3s 385ms/step - loss: 4.1966 - acc: 0.0425
Epoch 8/500
8/8 [==============================] - 3s 384ms/step - loss: 4.3269 - acc: 0.0613
Epoch 9/500
8/8 [==============================] - 3s 345ms/step - loss: 4.0482 - acc: 0.0629
Epoch 10/500
8/8 [==============================] - 3s 351ms/step - loss: 4.2329 - acc: 0.0825
Epoch 11/500
8/8 [==============================] - 3s 388m

I'll also evaluate the model's efficacy by running the test data through the NN.

In [ ]:
model.evaluate_generator(
      validation_generator,  
      )

[12.818301148394312, 0.7894737]

Some information I added after submission on the suggestion of my mentor:

I achieved an accuracy of 0.789 on my validation set, but no higher. To fix this, I tried:
- adding CNN layers
- changing the activation functions (relu to sigmoid helped but not a lot) of the fully connected layers
- changing the number of channels in the CNN layers
- adding fully connected layers and CNN layers
- decreasing the number of channels in the fully connected layers
- making my NN more shallow (I had a lot of layers at this point)
- increasing and decreasing the number of dropout layers
- increasing and decreasing the dropout/alpha rate
- changing the activation functions for the fully connected layers
- changing the number of the maxpool layers
- changing the image dimensions
- different ratios of validation and training set sizes (they changed, but not a lot)
- training for different numbers of epochs (50-500)

Given that all of these experiments didn’t improve the validation set accuracy much, I suspect that the accuracy of my model is limited by the size of my data set (less than 500).

Even though the test results are about 80%, I think it’s acceptable because (a) the data set is small and (b) random choice would have had only 2.5% accuracy. 